- 이번 과제의 목표인 아키텍처를 다루는데 집중해 봤습니다.
    - RNNs 구현을 잘 못해서 연습으로 붙여봤습니다.
    
    
- 데이터셋 중 택시, 버스, 영업용은 label의 한글로 나누었습니다.
    - 앞에 지역명이 붙은 label = 영업용 차량
    - [아/바/사/자]를 포함한 차량 번호 = 택시/버스
    
    
- 초록색 번호판은 패턴매칭을 통해서 제외시켰습니다.
    - blur + adaptive thresholding 후, 신규 번호판의 패턴에 맞는 번호판만 골라내었습니다.
    - 결과, [02버1520]을 제외하고 초록판을 모두 제외하였습니다.
    - 그러나, 신규 번호판도 10-20개 가량 제외되었습니다.
    - 개선안으로 초록색 detection 후, blur + adaptive thresholding을 적용할 예정입니다.
    
    
- 데이터셋 구성 후, Train과 Test로 나눈다음 loader를 구성했습니다.


- 모델은 CNN2RNN 아키텍처입니다.
    - 예정: test process 구현
    - 한계: local에서 작업해서 ... cpu가 열일을 했으나... 힘이 부족했습니다... 
    - google colab으로 작업 시, opencv가 한글을 인식못해서 전처리를 다시 해야해서 수정계획입니다.

In [1]:
import os
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms

#from torchvision.datasets import MNIST >>> 나중에 mnist pretrained 시도용
# from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset, DataLoader#, SubsetRandomSampler

In [2]:
def check_contour(w, h, area, ratio):
    MIN_AREA = 100
    MIN_WIDTH, MIN_HEIGHT = 5, 25
    MIN_RATIO, MAX_RATIO = 0.2, 0.5
    return area > MIN_AREA and w > MIN_WIDTH and h > MIN_HEIGHT and MIN_RATIO < ratio < MAX_RATIO

In [3]:
def is_green(img):
    height, width, channel = img.shape
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 가우시안 블러 + AdaptiveThresholding
    img_blurred = cv2.GaussianBlur(gray, ksize=(3, 3), sigmaX=0)
    img_blur_thresh = cv2.adaptiveThreshold(gray, maxValue=255.0,
                                           adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                           thresholdType=cv2.THRESH_BINARY_INV,
                                           blockSize=19,
                                           C=9)

    # 윤곽찾기 -> Pattern Matching
    # 가로 신축형 윤곽이 아니면 초록색(옛날 번호판)으로 구분
    contours, _ = cv2.findContours(img_blur_thresh,
                                  mode=cv2.RETR_LIST,
                                  method=cv2.CHAIN_APPROX_SIMPLE)
    contours_dict = []
    cnt = 0

    #idx = 0
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        ratio = w / h
        cx = x + (w / 2),
        cy = y + (h / 2)

        if check_contour(w, h, area, ratio):
            cnt += 1
            #name = 'conout' + str(idx)
            contours_dict.append({
            'contour': contour,
            'x1': x,
            'y1': y,
            'x2': x+w,
            'y2': y+h,
            'w': w,
            'h': h,
            'cx': cx,
            'cy': cy
            })

    if len(contours_dict) == 0:
        return True
    
    # 신형 번호판의 최소 만족 갯수 3개
    if cnt < 3:
        return True
    else:
        return False

    # bbox 중복제거용.
    # test1. 동작 X contours_dict를 우선순위 heap으로 만들고 while문으로 구현
    # test2. 초록색 탐지 -> 초록색 아닌걸로 판명난 걸 이 method로 탐지
    """idx = []
    for i in range(len(contours_dict)):
        box1 = contours_dict[i]

        for j in range(i+1, len(contours_dict)):
            box2 = contours_dict[j]

            x_overlap  = max(0, min(box1['x2'], box2['x2']) - max(box1['x1'], box2['x1']))
            y_overlap  = max(0, min(box1['y2'], box2['y2']) - max(box1['y1'], box2['y1']))
            overallaped_area = x_overlap * y_overlap

            if overallaped_area > 3:
                if box1['w']*box1['h'] < box2['w']*box2['h']:
                    del contours_dict[i]
                else:
                    del contours_dict[j]

    for i in range(len(idx)):
        cnt -= 1
        del contours_dict[idx[i][0]]"""

In [4]:
# 데이터 제외함수 (택시, 버스, 영업용, )
def exclude_data(file_list, path):
    tmp = [name for name in file_list if not any(re.findall(r'바|사|아|자', name))] # taxi 제외
    res = [name for name in tmp if not any(re.findall('[가-힣]+', name[0:2]))]  # 앞에 지역명을 갖고있는 영업용차량 제외

    valid_data = []
    for name in tqdm(res):

        img_pil = Image.open(path+name)
        img_np = np.array(img_pil)
        img = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

        if not is_green(img):
            valid_data.append(name)
    return valid_data

In [5]:
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [6]:
if __name__=='__main__':

    path = 'C:/Users/anjae/Desktop/PerspectiveImages/'
    file_list = os.listdir(path)
    valid_data = exclude_data(file_list, path)    # >>> 812

100%|███████████████████████████████████████████████████████████████████████████████| 981/981 [00:01<00:00, 773.90it/s]


In [7]:
captions = [name[:-4] for name in valid_data] # eliminate the .jpg
counter = Counter()
for cap in captions:
    for token in cap:
        counter.update(token)
        
words = [word for word, cnt in counter.items()]
vocab = Vocabulary()
vocab.add_word('<start>')
vocab.add_word('<end>')

for i, word in enumerate(words):
    vocab.add_word(word)

In [8]:
vocab.idx2word

{0: '<start>',
 1: '<end>',
 2: '0',
 3: '1',
 4: '도',
 5: '9',
 6: '8',
 7: '2',
 8: '두',
 9: '6',
 10: '7',
 11: '3',
 12: '오',
 13: '호',
 14: '4',
 15: '거',
 16: '모',
 17: '버',
 18: '5',
 19: '주',
 20: '고',
 21: '나',
 22: '라',
 23: '로',
 24: '루',
 25: '보',
 26: '부',
 27: '가',
 28: '마',
 29: '머',
 30: '소',
 31: '조',
 32: '하',
 33: '구',
 34: '누',
 35: '러',
 36: '노',
 37: '수',
 38: '우',
 39: '무',
 40: '저',
 41: '더',
 42: '너',
 43: '어',
 44: '다',
 45: '서'}

In [9]:
vocab.word2idx

{'<start>': 0,
 '<end>': 1,
 '0': 2,
 '1': 3,
 '도': 4,
 '9': 5,
 '8': 6,
 '2': 7,
 '두': 8,
 '6': 9,
 '7': 10,
 '3': 11,
 '오': 12,
 '호': 13,
 '4': 14,
 '거': 15,
 '모': 16,
 '버': 17,
 '5': 18,
 '주': 19,
 '고': 20,
 '나': 21,
 '라': 22,
 '로': 23,
 '루': 24,
 '보': 25,
 '부': 26,
 '가': 27,
 '마': 28,
 '머': 29,
 '소': 30,
 '조': 31,
 '하': 32,
 '구': 33,
 '누': 34,
 '러': 35,
 '노': 36,
 '수': 37,
 '우': 38,
 '무': 39,
 '저': 40,
 '더': 41,
 '너': 42,
 '어': 43,
 '다': 44,
 '서': 45}

In [10]:
captions

['01도0982',
 '01두6167',
 '01두6173',
 '01오2026',
 '01호7971',
 '01호9428',
 '02거0688',
 '02모6182',
 '02버1520',
 '02버8171',
 '02주8096',
 '03고2103',
 '03나5394',
 '03라8025',
 '03로2945',
 '03루5237',
 '03버4113',
 '03보8129',
 '03부4568',
 '04가2678',
 '04고8608',
 '04나1851',
 '04마3810',
 '04머0711',
 '04소0381',
 '04소0425',
 '04조5542',
 '04하0493',
 '05가7190',
 '05거0214',
 '05고3250',
 '05구8036',
 '05누6006',
 '05누6109',
 '05누6625',
 '05도1291',
 '05도7067',
 '05라9211',
 '05로6033',
 '05모9326',
 '05소4660',
 '06러9201',
 '06러9329',
 '06루2679',
 '06모1085',
 '06모1297',
 '06모1437',
 '07고4598',
 '07노9076',
 '07두4907',
 '07모3479',
 '07소3599',
 '07수1184',
 '08두5232',
 '08보1242',
 '08보1292',
 '08수7006',
 '08수7040',
 '08우1024',
 '09누6636',
 '09라0676',
 '09라0710',
 '09라0740',
 '09라0798',
 '09러2342',
 '09러2376',
 '09마0604',
 '09머5540',
 '09무3054',
 '09우5531',
 '09저6947',
 '10누5019',
 '10누5053',
 '10더5037',
 '10루4154',
 '10무1590',
 '10부6770',
 '10소2938',
 '10우3213',
 '10조8410',
 '10주0605',
 '11너9153',
 '11노3042',
 '11

In [11]:
# fInd mean and std for each channel for valid_data
imgs = []
for file_name in valid_data:
    img = Image.open(path + file_name)
    imgs.append(np.asarray(img))

imgs = np.asarray(imgs)
normalize_mean = [imgs[:,:,:,0].mean(), imgs[:,:,:,1].mean(), imgs[:,:,:,2].mean()]
normalize_std = [imgs[:,:,:,0].std(), imgs[:,:,:,1].std(), imgs[:,:,:,2].std()]

In [12]:
normalize_mean, normalize_std

([140.35294402260286, 141.85065142150805, 140.9446750838778],
 [67.1339938663095, 66.77541666677854, 66.45039705287245])

In [13]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(normalize_mean,normalize_std),
])

In [14]:
class CustomDataset(Dataset):
    def __init__(self, imgs, vocab, captions, transform):
        self.imgs = imgs
        self.captions = captions
        self.vocab = vocab
        self.transform = transform
        
    def __getitem__(self, index):
        imgs = self.imgs
        vocab = self.vocab
        captions = self.captions
        
        image = imgs[index]
        if self.transform is not None:
            image = self.transform(image)
        
        tokens = [token for token in captions[index]]
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        target = torch.Tensor(caption)
        
        return image, target
    
    def __len__(self):
        return len(self.imgs)

In [15]:
batch_size = 8
shuffle = False

c = CustomDataset(imgs, vocab, captions, transform)
data_loder = DataLoader(dataset=c,
                       batch_size=batch_size,
                       shuffle=shuffle,
                       )

train_size = int(0.8 * c.__len__())
test_size = c.__len__() - train_size

In [16]:
torch.manual_seed(10)
train_dataset, test_dataset = data.random_split(c, [train_size, test_size])
train_loder = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loder = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

In [17]:
# Mnist데이터 훈련된 vgg11 모델이 필요하다
# 나중에 업데이트
class MnistVgg11():
    def __init__(self):
        self.vgg11 = models.vgg11(pretrained=True).features
        self.train_batch_size = 16
        train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=train_batch_size, shuffle=True)
        output = self.vgg1(train_loader.sampler.data_source)
        self.vgg11.classifier = nn.Linear(output, 10)
    
    def train(self):
        epoch = 100

In [18]:
# Image captionning approach
class EncoderCNN(nn.Module):
    def __init__(self, input_dim, embed_size):
        super(EncoderCNN, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(input_dim, 32, kernel_size=3, stride=1),
                                  nn.ReLU(),
                                  nn.BatchNorm2d(32),
                                  nn.Conv2d(32, 64, kernel_size=3, stride=1),
                                  nn.ReLU(),
                                  nn.BatchNorm2d(64),
                                  nn.Conv2d(64, 128, kernel_size=3, stride=1),
                                  nn.ReLU(),
                                  nn.BatchNorm2d(128),
                                  nn.Conv2d(128, 128, kernel_size=3, stride=1),
                                  nn.ReLU(),
                                  nn.AdaptiveAvgPool2d((7,7))
                                 )
        self.linear = nn.Linear(6272, embed_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, images):
        features = self.conv(images)
  
        return self.dropout(self.relu(self.linear(features.reshape(images.shape[0], -1))))

    
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1, batch_first=True):
        super(DecoderRNN, self).__init__()
        self.hidden_dim = hidden_size
        self.num_layers = num_layers
        self.embed = nn.Embedding(vocab_size, embed_size)  # 46 x 256
        self.lstm = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, \
                            num_layers=num_layers, batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, feature, caption):
        """CNN으로 얻은 feature 벡터를 복호화하고 캡션을 생성"""
        
        # batch x seq_length x embed_size
        # 8 x 8 x 256
        x = self.embed(caption) 
        
        # batch x num_layers*num_directions x embed_size
        # 8 x 1 x 256
        h0 = feature
        c0 = torch.zeros_like(h0)
        
        
        out, _ = self.lstm(x, (h0, c0))
        output = self.linear(out)
        return output

class CNN2RNN(nn.Module):
    def __init__(self, input_dim, embed_size, hidden_size, vocab_size, num_layers=1, batch_first=True):
        super(CNN2RNN, self).__init__()
        self.encoder = EncoderCNN(input_dim, embed_size)
        self.decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers, batch_first=batch_first)
        self.batch_first = batch_first
        
    def forward(self, x, caption):
        feature = self.encoder(x)    # 8 x 256
        feature_ = feature.unsqueeze(0)    # 8 x 1 x 256

        caption_ = caption[:,:-1]    # 8 x 9 -> 8 x 8
        output = self.decoder(feature_, caption_)
        return output
    
    def sample_caption(self, image, vocab):
        result = []
        
        with torch.no_grad():
            if len(image.shape) < 4:
                image = image.unsqueeze(0)
            feature = self.encoder(image).unsqueeze(0) # image를 한개씩 받기 때문에 batch차원 늘려준다
            states = None
            hidden = feature
            
            for _ in range(8):
                hiddens, states = self.decoder.lstm(hidden, states)
                output = self.decoder.linear(hiddens).squeeze(0)
                predicted = output.argmax(1)
                result.append(predicted.item())
                x = self.decoder.embed(predicted).unsqueeze(0)
                hidden = hiddens
                
                if vocab.idx2word[predicted.item()] == '<end>':
                    break
                    
        return [vocab.idx2word[i] for i in result]

In [42]:
def save_checkpoint(state, epo, filename='my_checkpoint'):
    print(">>> saving checkpoing")
    torch.save(state, filename + str(epo) + '.path.tar')
    
def load_checkpoint(checkpoint):
    model.load_state_dict(torch.load(checkpoint)['state_dict'])
    optimizer.load_state_dict(torch.load(checkpoint)['optimizer'])

In [50]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.003
epoch = 100
embed_size = 256
hidden_size = 256
vocab_size = len(vocab)
input_dim = train_dataset.dataset.imgs.shape[-1]
model = CNN2RNN(input_dim, embed_size, hidden_size, len(vocab))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

epo = 3    # custom number for what epoch model and optimizer 
checkpoint = 'my_checkpoint3.path.tar'
load_model = True

In [ ]:
if load_model:
    load_checkpoint(checkpoint)

for img, caption in test_loder:
    sample_img = img[0]
    sample_caption = caption[0]
    break
    
for epo in range(4, epoch):
    running_loss = 0
    
    model.train()
    for idx, (image, caption) in tqdm(enumerate(train_loder)):
        image = image.to(device)
        caption = caption.type(torch.LongTensor).to(device)

        output = model(image, caption)   
        result = output.reshape(-1, output.shape[2])
        target = caption[:, 1:].reshape(-1)
        loss = criterion(result, target)
    
        predicted = [each.argmax(1) for each in output]
        predicted_ = torch.stack(predicted)
        correct = caption[:, 1:]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    print(f'epoch: {epo}, loss: {running_loss}')
    checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    save_checkpoint(checkpoint, epo)
    
    model.eval()
    with torch.no_grad():
        res = model.sample_caption(sample_img, vocab)
        print(res)
        for each in sample_caption:
            print(vocab.idx2word[each.item()], end='')
        print('\n')

81it [08:42,  6.44s/it]
0it [00:00, ?it/s]

epoch: 4, loss: 148.75156569480896
>>> saving checkpoing
['4', '<end>']
<start>06모1297<end>



81it [08:30,  6.30s/it]
0it [00:00, ?it/s]

epoch: 5, loss: 141.91908645629883
>>> saving checkpoing
['루', '<end>']
<start>06모1297<end>



81it [08:29,  6.30s/it]
0it [00:00, ?it/s]

...

81it [08:06,  6.00s/it]
0it [00:00, ?it/s]

epoch: 37, loss: 76.82195454835892
>>> saving checkpoing
['노', '라', '나', '나', '8', '오', '<end>']
<start>06모1297<end>



81it [09:32,  7.07s/it]
0it [00:00, ?it/s]

epoch: 38, loss: 76.85344737768173
>>> saving checkpoing
['너', '라', '나', '8', '8', '8', '<end>']
<start>06모1297<end>



71it [08:26, 17.04s/it]

- 결과가 처음에 2번째 step만에 끝내버리던 것이 step을 점점 더 잘 보고 키우고있습니다.

- sample test결과가 안좋은 이유에 대해서는 2가지 개선 예정 입니다.
    - CNN을 pretrained 시켜서 적용해보는 방법
    - CNN의 구조를 fancy하게 바꾸는 방법

- 30 epoch만에 2 step-> 6 step으로 왔는데 개선이 필요해한 것 같습니다.

- 오늘 study중 구현 방법에서 CNN 으로만 구현, feature map의 channel을 seq_length에 맞춰서 예측해보는 방법도 적용해 볼 예정입니다.